# Install nodejs
- https://medium.com/nectec/node-js-grpc-639729d5e545
- https://medium.com/nectec/node-js-grpc-streaming-7ac9b1e1090e


In [ ]:
! apt install nodejs

! node -v

! apt update
! apt install npm
! npm -v

# intial project

In [ ]:
! npm init -y

# Create package.json

In [ ]:
%%writefile package.json
{
  "name": "gRPC",
  "version": "1.0.0",
  "description": "",
  "main": "server.js",
  "scripts": {
    "start": "node server.js"
  },
  "keywords": [],
  "author": "",
  "license": "ISC",
  "dependencies": {
    "@grpc/grpc-js": "^1.3.2",
    "@grpc/proto-loader": "^0.5.0",
    "async": "^3.2.0",
    "google-protobuf": "^3.6.1",
    "node-grpc-client": "^1.5.0"
  }
}


# Install package

In [1]:
! npm install

npm WARN deprecated grpc@1.24.11: This library will not receive further updates other than security fixes. We recommend using @grpc/grpc-js instead.
npm WARN deprecated glob@7.2.3: Glob versions prior to v9 are no longer supportedm
npm WARN deprecated rimraf@3.0.2: Rimraf versions prior to v4 are no longer supported
npm WARN deprecated npmlog@5.0.1: This package is no longer supported.
npm WARN deprecated inflight@1.0.6: This module is not supported, and leaks memory. Do not use it. Check out lru-cache if you want a good and tested way to coalesce async requests by a key value, which is much more comprehensive and powerful.
npm WARN deprecated gauge@3.0.2: This package is no longer supported.[Kwid
npm WARN deprecated are-we-there-yet@2.0.0: This package is no longer supported.
        ......] - build:long: sill linkStuff long@5.2.3 has /opt/data/WebDm: T
> grpc@1.24.11 install /opt/data/WebDev/gRPC/nodeJS/node_modules/grpc
> node-pre-gyp install --fallback-to-build --library=static_lib

# Server

In [5]:
%%writefile server.js
var PROTO_PATH = "../protos/chatbot.proto";

var grpc = require("@grpc/grpc-js");
var protoLoader = require("@grpc/proto-loader");
var packageDefinition = protoLoader.loadSync(PROTO_PATH, {
  keepCase: true,
  longs: String,
  enums: String,
  defaults: true,
  oneofs: true,
});
var chat_proto = grpc.loadPackageDefinition(packageDefinition).chatPack;

function ChatFunc(call, cb_Reply) {
  
  // console.log("ChatFunc:", call.metadata.get("apikey"));
  console.log("ChatFunc:", call.request.name);
  cb_Reply(null, { result: "Hello " + call.request.name });
}
function ChatStream(call) {
  console.log("ChatStream:");
  call.on("data", function (st) {
    console.log("data:", st.name);
    for (let x = 0; x < 5; x++) {
      const dataToSend = {
        name: "blue",
        result: x,
      };
      call.write(dataToSend);
    }
    call.end();
  });
}

function main() {
  var server = new grpc.Server();
  server.addService(chat_proto.ChatService.service, {
    ChatFunc: ChatFunc,
    ChatStream: ChatStream,
  });
  server.bindAsync(
    "0.0.0.0:8099",
    grpc.ServerCredentials.createInsecure(),
    () => {
      server.start();
    }
  );
}

main();

Overwriting server.js


# Client

In [1]:
%%writefile client.js
var PROTO_PATH = "../protos/chatbot.proto";

var grpc = require("@grpc/grpc-js");
var protoLoader = require("@grpc/proto-loader");
var packageDefinition = protoLoader.loadSync(PROTO_PATH, {
  keepCase: true,
  longs: String,
  enums: String,
  defaults: true,
  oneofs: true,
});
var chat_proto = grpc.loadPackageDefinition(packageDefinition).chatPack;
const dataToSend = {
  name: "blue",
  result: 0,
};
function main() {
  var client = new chat_proto.ChatService(
    "localhost:8099",
    grpc.credentials.createInsecure()
  );

  var user = "blue";
  var meta = new grpc.Metadata();
  meta.add('apikey', '12345678');
  client.ChatFunc({ name: user }, meta,function (err, response) {
    console.log("Service response :", response.result);
  });
}
console.log("begin");
main();
console.log("end");


Overwriting client.js


In [2]:
! node client

begin
end
Service response : Hello blue


In [6]:
%%writefile streamClient.js
var PROTO_PATH = "../protos/chatbot.proto";

var grpc = require("@grpc/grpc-js");
var protoLoader = require("@grpc/proto-loader");
var packageDefinition = protoLoader.loadSync(PROTO_PATH, {
  keepCase: true,
  longs: String,
  enums: String,
  defaults: true,
  oneofs: true,
});
var chat_proto = grpc.loadPackageDefinition(packageDefinition).chatPack;
const dataToSend = {
  name: "blue",
  result: 0,
};
function main() {
  var client = new chat_proto.ChatService(
    "localhost:8099",
    grpc.credentials.createInsecure()
  );
  var call = client.ChatStream();

  call.write(dataToSend);
  call.on("data", function (st) {
    console.log("Service stream :", st.result);
  });
  call.on("end", function () {
    console.log("end:");
  });
}
console.log("begin");
main();
console.log("end");


Overwriting streamClient.js


In [7]:
! node streamClient

begin
end
Service stream : 0
Service stream : 1
Service stream : 2
Service stream : 3
Service stream : 4
end:
